<a href="https://colab.research.google.com/github/yvoirin/python_notebooks/blob/main/Google_Earth_Engine_dans_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Utiliser Google Earth Engine dans Colab

## Installer EE

Il faut importer ee dans Colab. Ensuite il faut autoriser Google Earth Engine dans Colab.

In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=csXhkS2rbXitu9UnxiuWC3ql4NDroCsneY2TiCadFvs&tc=DIVeS19Lu9TvgiS_ZvNCPKKiSxR0SCYaL9SHPujSPJI&cc=3sbxPgJRFzFbkSDKcGrUb_puF6JSeGjeBkclYuv03_4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh6HM2E4tnE_YZYQKch0sw1VzTOnhHIpALP-yGo1xAFLleK7t5DuH8M

Successfully saved authorization token.


## Chercher un jeu de données

Vous pouvez aller voir les jeux de données dans : https://developers.google.com/earth-engine/

Si on souhaite utiliser les données Landsat 8 (USGS Landsat 8 Collection 2 Tier 1 TOA Reflectance), on peut utiliser le jeu suivant : LANDSAT/LC08/C02/T1_TOA

In [32]:
# On charge le jeu de données

images = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')

# on définit un point d'intérêt (ex: Sherbrooke)
xy = ee.Geometry.Point([-71.888351, 45.404476])

# On peut définir un buffer autour de ce point pour viser plusieurs images
buffer = xy.buffer(0.01)
# On peut filtrer par dates aussi
sherbrooke_images = images.filterBounds(buffer).filterDate('2019-05-01','2019-07-30').sort('CLOUD_COVER')
# les images disponibles seront dans une liste, on va les trier par la couverture nuageuse
# et on prendra la première
scene = sherbrooke_images.first()


## Affichage des images

On peut afficher les images dans folium, il faut toutefois définir une fonction pour charger les images

In [33]:
import folium

# on utilise la fonction suivante
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# on définit la méthode pour ouvrir les couches EE
folium.Map.add_ee_layer = add_ee_layer

# on définit des paramètres pour visualiser notre image
vis_params = {'bands': ['B4', 'B3', 'B2'], 'max': 0.3}
# on va créer une carte centrée sur Sherbrooke
my_map = folium.Map(location=[45.404476,-71.888351], zoom_start=12)

# on affiche l'image
my_map.add_ee_layer(scene, vis_params, 'Sherbrooke')
# on ajoute les contrôles pour la carte
my_map.add_child(folium.LayerControl())

# on affiche la carte
display(my_map)

## Petit traitement

On peut appliquer un filtre sur l'image et l'afficher

In [36]:
import folium

# on utilise la fonction suivante
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# on définit la méthode pour ouvrir les couches EE
folium.Map.add_ee_layer = add_ee_layer

# on définit des paramètres pour visualiser notre image
vis_params = {'bands': ['B4', 'B3', 'B2'], 'max': 0.3}
# on va créer une carte centrée sur Sherbrooke
my_map = folium.Map(location=[45.404476,-71.888351], zoom_start=12)

# on définit les valeurs du filtre
weights = [
    [1,1,1,1,1],
    [1,1,1,1,1],
    [1,1,-24,1,1],
    [1,1,1,1,1],
    [1,1,1,1,1]
]
# on assemble le filtre
filter = ee.Kernel.fixed(5, 5, weights, -2, -2, False)
# on applique la convolution
imagefiltered = scene.convolve(filter)

my_map.add_ee_layer(imagefiltered, vis_params, 'Sherbrooke')

my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)